In [1]:
"""
this is the 3rd ipynb
it can create catalog with MGS, dataset, should after data_augmentation.py and data_scale.py
"""

'\nthis is the 3rd ipynb\nit can create catalog with MGS, dataset\n'

In [1]:
import pandas as pd
from pandas import read_parquet
import os
from astropy.io import fits
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import  tqdm
from sklearn.model_selection import train_test_split
def match(df_1, df_2, pixel, df1_name):
    """
    match two catalog
    :param df_1:
    :param df_2:
    :return:
    """
    sdss = SkyCoord(ra=df_1.ra * u.degree, dec=df_1.dec * u.degree)
    decals = SkyCoord(ra=df_2.ra * u.degree, dec=df_2.dec * u.degree)
    idx, d2d, d3d = sdss.match_to_catalog_sky(decals)
    max_sep = pixel * 0.262 * u.arcsec
    distance_idx = d2d < max_sep
    sdss_matches = df_1.iloc[distance_idx]
    matches = idx[distance_idx]
    decal_matches = df_2.iloc[matches]
    test = sdss_matches.loc[:].rename(columns={"ra":"%s" % df1_name[0], "dec":"%s" % df1_name[1]})
    test.insert(0, 'ID', range(len(test)))
    decal_matches.insert(0, 'ID', range(len(decal_matches)))
    new_df = pd.merge(test, decal_matches, how="inner", on=["ID"])
    return new_df.drop("ID", axis=1)

In [3]:
# https://doi.org/10.5281/zenodo.4573248
# arXiv:2102.08414v2
# here is Mike Walmsley's catalog
df_auto = read_parquet("/data/renhaoye/MorCG/dataset/gz_decals_auto_posteriors.parquet").rename(columns=lambda x: x.replace("-", "_")) # 头中的-换为_
len(df_auto)

313789

In [2]:
# main galaxy sample raw catalog, 74w sample
with fits.open("/data/renhaoye/MorCG/dataset/VAGC_MGS-m14_1777--20180116.fits") as hdul:
    ra = np.array(hdul[1].data["RA      "])
    dec = np.array(hdul[1].data["DEC     "])
    petro_mag = np.array(hdul[1].data["PETROMAG"][:, 2])
    MGS = pd.DataFrame(np.array((ra, dec, petro_mag)).T, columns=["ra", "dec", "petro_mag"])

In [4]:
# load local file names to pd.Dataframe to match, in_decals, ls-9-south
# these data are in MGS
local_files = os.listdir("/data/renhaoye/MorCG/dataset/in_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
in_decals = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359209/359209 [00:00<00:00, 466612.55it/s]


In [5]:
# load local file names to pd.Dataframe to match, out_decals, ls-9-north
# these data are in MGS
local_files = os.listdir("/data/renhaoye/MorCG/dataset/out_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
out_decals = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 259323/259323 [00:00<00:00, 477927.14it/s]


In [6]:
# overlap = galaxy both in DECaLS and BASS+MzLS, 62497 galaxies
overlap = match(in_decals, out_decals, 2, df1_name=["in_ra", "in_dec"]).drop(columns=["in_ra", "in_dec"])

In [8]:
# from tqdm import  tqdm
# # load local file names to pd.Dataframe to match
# local_files = os.listdir("/data/GZ_Decals/nomerge/")
# ra, dec = [], []
# for i in tqdm(range(len(local_files))):
#     if ".fits" in local_files[i]:
#         t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
#         ra.append(t_ra)
#         dec.append(t_dec)
# local_files = os.listdir("/data/GZ_Decals/merge/")
# for i in tqdm(range(len(local_files))):
#     if ".fits" in local_files[i]:
#         t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
#         ra.append(t_ra)
#         dec.append(t_dec)
# local_dir = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

In [7]:
# use MikeWalmsley's catalog to match Main Galaxy Sample catalog and add petro_mag info
df_auto_mgs = match(MGS, df_auto, 5, ["MGS_ra", "MGS_dec"])

In [8]:
df_auto_mgs

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
0,337.450310,1.266134,16.952042,337.450318,1.266150,J222948.07+011558.1,4.186915,1.915174,4.839483,-20.407906,...,0.018522,0.018522,0.488956,0.451788,0.003392,0.003392,1.0,dr5/png/J222/J222948.07+011558.1.png,146.525273,False
1,343.100541,1.271833,17.530886,343.100545,1.271857,J225224.12+011618.6,3.553056,1.755761,4.028279,-18.125319,...,0.072188,0.072188,0.824894,0.013147,0.007565,0.007565,1.0,dr5/png/J225/J225224.12+011618.6.png,151.577305,False
2,350.324605,1.269114,17.309258,350.324592,1.269100,J232117.90+011608.8,4.028279,1.842611,4.394024,-21.143169,...,0.103261,0.103261,0.810387,0.008353,0.003222,0.003222,1.0,dr5/png/J232/J232117.90+011608.8.png,138.786222,False
3,353.543418,1.271758,17.572227,353.543420,1.271777,J233410.42+011618.4,7.190163,3.434981,7.509485,-20.800541,...,0.476523,0.476523,0.435209,0.015857,0.355152,0.355152,1.0,dr5/png/J233/J233410.42+011618.4.png,148.026903,False
4,353.910231,1.264805,17.464956,353.910235,1.264773,J233538.45+011553.2,7.995928,4.501389,13.797813,-20.697124,...,0.238907,0.238907,0.610592,0.049025,0.042594,0.042594,1.0,dr5/png/J233/J233538.45+011553.2.png,150.268244,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253235,161.961011,9.860375,16.593317,161.960998,9.860361,J104750.63+095137.3,9.472872,3.862174,12.797614,-21.275429,...,0.133771,0.133771,0.523560,0.209438,0.014531,0.014531,1.0,dr5/png/J104/J104750.63+095137.3.png,152.649336,False
253236,162.079082,10.040918,16.656054,162.079057,10.040934,J104818.97+100227.3,3.989568,1.789999,4.746917,-20.202862,...,0.084888,0.084888,0.836097,0.007511,0.003354,0.003354,1.0,dr5/png/J104/J104818.97+100227.3.png,148.478420,False
253237,162.179214,10.066991,16.888079,162.179191,10.067010,J104843.00+100401.1,3.639873,1.755761,4.436659,-18.020481,...,0.097034,0.097034,0.807933,0.005390,0.025462,0.025462,1.0,dr5/png/J104/J104843.00+100401.1.png,152.229218,False
253238,162.439221,9.985815,17.519909,162.439203,9.985830,J104945.40+095908.9,3.819922,1.697413,3.746857,-20.513891,...,0.112341,0.112341,0.814916,0.005653,0.020147,0.020147,1.0,dr5/png/J104/J104945.40+095908.9.png,154.227963,False


In [9]:
# this is local file both in auto catalog and MGS, in DECaLS region
# this is the final dataset
df_loc_decals = match(in_decals, df_auto_mgs, 2, ["del_ra", "del_dec"]).drop(columns=["del_ra", "del_dec"])

In [10]:
df_loc_decals

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
0,156.404678,32.128263,15.143953,156.404700,32.128272,J102537.12+320741.7,7.807667,3.405226,11.521922,-20.225849,...,0.095087,0.095087,0.609809,0.175285,0.005414,0.005414,1.0,dr5/png/J102/J102537.12+320741.7.png,116.723694,False
1,156.028195,10.281041,15.274831,156.028187,10.281041,J102406.76+101651.7,6.501150,2.819223,9.270042,-22.320599,...,0.096528,0.096528,0.777067,0.007275,0.003151,0.003151,1.0,dr5/png/J102/J102406.76+101651.7.png,111.390991,False
2,41.000599,-8.739704,16.954834,41.000604,-8.739700,J024400.14-084422.9,3.856986,1.878542,4.701300,-18.766365,...,0.038196,0.038196,0.649711,0.275622,0.006129,0.006129,1.0,dr5/png/J024/J024400.14-084422.9.png,140.838770,False
3,236.876370,0.425387,16.856783,236.876419,0.425334,J154730.33+002531.4,8.806542,4.248001,9.377006,-18.320585,...,0.386491,0.386491,0.525614,0.013837,0.240117,0.240117,1.0,dr5/png/J154/J154730.33+002531.4.png,152.437658,False
4,233.329425,7.594146,17.295204,233.329489,7.594180,J153319.06+073538.9,3.434981,1.563657,3.970353,-19.721277,...,0.044695,0.044695,0.797498,0.108880,0.004655,0.004655,1.0,dr5/png/J153/J153319.06+073538.9.png,154.238022,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171362,141.485652,7.017201,17.068510,141.485663,7.017227,J092556.55+070101.9,3.336901,1.497167,4.106831,-20.020388,...,0.092294,0.092294,0.827177,0.024317,0.005223,0.005223,1.0,dr5/png/J092/J092556.55+070101.9.png,150.509885,False
171363,138.774497,3.585291,17.036930,138.774441,3.585325,J091505.87+033507.0,9.129296,4.975659,10.416905,-21.397547,...,0.212240,0.212240,0.600900,0.011914,0.020481,0.020481,1.0,dr5/png/J091/J091505.87+033507.0.png,151.126267,False
171364,129.135154,27.277870,17.037573,129.135177,27.277865,J083632.43+271640.3,5.566817,2.943241,5.648035,-20.498346,...,0.625170,0.625170,0.306107,0.023183,0.521136,0.521136,1.0,dr5/png/J083/J083632.43+271640.3.png,152.656195,False
171365,134.269900,30.152608,17.012079,134.269932,30.152604,J085704.78+300909.3,6.309475,3.193874,7.670071,-20.244175,...,0.680549,0.680549,0.227615,0.034112,0.560014,0.560014,1.0,dr5/png/J085/J085704.78+300909.3.png,152.301367,False


In [11]:
overlap_local = match(overlap, df_loc_decals, 2, ["del_ra", "del_dec"]).drop(columns=["del_ra", "del_dec"])

In [12]:
overlap_local

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
0,156.404678,32.128263,15.143953,156.404700,32.128272,J102537.12+320741.7,7.807667,3.405226,11.521922,-20.225849,...,0.095087,0.095087,0.609809,0.175285,0.005414,0.005414,1.0,dr5/png/J102/J102537.12+320741.7.png,116.723694,False
1,353.325279,0.987352,17.009586,353.325280,0.987377,J233318.06+005914.4,2.171330,1.047382,2.750871,-20.270401,...,0.060103,0.060103,0.523389,0.340287,0.005432,0.005432,1.0,dr1_dr2/png/dr1/standard/J233318.06+005914.4_s...,143.753298,False
2,353.134865,0.648646,17.057093,353.134896,0.648666,J233232.37+003855.1,5.842183,2.929065,6.465595,-19.588432,...,0.445478,0.445478,0.482556,0.014868,0.364242,0.364242,1.0,dr5/png/J233/J233232.37+003855.1.png,136.221183,False
3,37.034101,0.054160,16.462898,37.034140,0.054153,J022808.18+000314.9,8.390046,3.923810,9.933681,-20.116978,...,0.334067,0.334067,0.345201,0.272013,0.253404,0.253404,1.0,dr5/png/J022/J022808.18+000314.9.png,146.234857,False
4,41.180459,1.222984,17.606537,41.180461,1.222956,J024443.30+011322.6,3.639873,1.578830,4.394024,-19.227112,...,0.124096,0.124096,0.776697,0.032473,0.018239,0.018239,1.0,dr5/png/J024/J024443.30+011322.6.png,149.439550,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294,38.324794,-1.246896,17.740160,38.324793,-1.246923,J023317.95-011448.4,10.377614,4.394024,10.682637,-18.379513,...,0.010711,0.010711,0.400311,0.549677,0.002765,0.002765,1.0,dr5/png/J023/J023317.95-011448.4.png,152.148657,False
4295,170.880584,32.458464,15.705607,170.880616,32.458477,J112331.34+322730.4,7.587428,3.385794,11.456174,-21.369747,...,0.114476,0.114476,0.624014,0.202886,0.007930,0.007930,1.0,dr5/png/J112/J112331.34+322730.4.png,150.005443,False
4296,39.218964,-0.955296,17.715490,39.218978,-0.955312,J023652.55-005719.0,3.015157,1.333357,3.970353,-20.233921,...,0.092765,0.092765,0.810172,0.013194,0.007430,0.007430,1.0,dr5/png/J023/J023652.55-005719.0.png,126.138830,False
4297,227.203124,32.863581,17.088730,227.203096,32.863555,J150848.74+325148.8,3.385586,1.526362,4.746917,-20.884190,...,0.101822,0.101822,0.783381,0.047958,0.006626,0.006626,1.0,dr5/png/J150/J150848.74+325148.8.png,148.490371,False


In [21]:
overlap_local.to_csv("/data/renhaoye/MorCG/dataset/overlap_local.csv")

In [13]:
threshold = 0.5
threshold2 = 0.6
merger = df_loc_decals.query('merging_minor_disturbance_fraction > %f  | merging_major_disturbance_fraction > %f | merging_merger_fraction > %f '
                             % (threshold2, threshold2, threshold2))
round = df_loc_decals.query('smooth_or_featured_smooth_fraction >  %f  & how_rounded_round_fraction > %f & merging_none_fraction > %f'
                            % (0.7, 0.8, threshold2))
between = df_loc_decals.query('smooth_or_featured_smooth_fraction >  %f & how_rounded_in_between_fraction > %f & merging_none_fraction > %f'
                              % (0.7, 0.85, threshold2))
cigar = df_loc_decals.query('smooth_or_featured_smooth_fraction > %f & how_rounded_cigar_shaped_fraction > %f & merging_none_fraction > %f'
                            % (threshold, threshold2, threshold2))
edgeOn = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_yes_fraction > %f & merging_none_fraction > %f'
                             % (threshold, 0.7, threshold2))
noBar = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_no_fraction > %f & bar_no_fraction > %f  & merging_none_fraction > %f'
                            % (threshold, threshold, 0.7, threshold2))
strongBar = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_no_fraction > %f &bar_strong_fraction > %f & merging_none_fraction > %f'
                                % (threshold, threshold, 0.55, threshold2))

merger.insert(loc=7, column='label', value=0)
round.insert(loc=7, column='label', value=1)
between.insert(loc=7, column='label', value=2)
cigar.insert(loc=7, column='label', value=3)
edgeOn.insert(loc=7, column='label', value=4)
noBar.insert(loc=7, column='label', value=5)
strongBar.insert(loc=7, column='label', value=6)

merger_train, merger_test = train_test_split(merger, test_size=0.1, random_state=1926)
merger_train, merger_valid = train_test_split(merger_train, test_size=0.1, random_state=1926)
merger_train.insert(loc=7, column='func', value="train")
merger_valid.insert(loc=7, column='func', value="valid")
merger_test.insert(loc=7, column='func', value="test")

round_train, round_test = train_test_split(round, test_size=0.1, random_state=1926)
round_train, round_valid = train_test_split(round_train, test_size=0.1, random_state=1926)
round_train.insert(loc=7, column='func', value="train")
round_valid.insert(loc=7, column='func', value="valid")
round_test.insert(loc=7, column='func', value="test")

between_train, between_test = train_test_split(between, test_size=0.1, random_state=1926)
between_train, between_valid = train_test_split(between_train, test_size=0.1, random_state=1926)
between_train.insert(loc=7, column='func', value="train")
between_valid.insert(loc=7, column='func', value="valid")
between_test.insert(loc=7, column='func', value="test")

cigar_train, cigar_test = train_test_split(cigar, test_size=0.1, random_state=1926)
cigar_train, cigar_valid = train_test_split(cigar_train, test_size=0.1, random_state=1926)
cigar_train.insert(loc=7, column='func', value="train")
cigar_valid.insert(loc=7, column='func', value="valid")
cigar_test.insert(loc=7, column='func', value="test")

edgeOn_train, edgeOn_test = train_test_split(edgeOn, test_size=0.1, random_state=1926)
edgeOn_train, edgeOn_valid = train_test_split(edgeOn_train, test_size=0.1, random_state=1926)
edgeOn_train.insert(loc=7, column='func', value="train")
edgeOn_valid.insert(loc=7, column='func', value="valid")
edgeOn_test.insert(loc=7, column='func', value="test")

noBar_train, noBar_test = train_test_split(noBar, test_size=0.1, random_state=1926)
noBar_train, noBar_valid = train_test_split(noBar_train, test_size=0.1, random_state=1926)
noBar_train.insert(loc=7, column='func', value="train")
noBar_valid.insert(loc=7, column='func', value="valid")
noBar_test.insert(loc=7, column='func', value="test")

strongBar_train, strongBar_test = train_test_split(strongBar, test_size=0.1, random_state=1926)
strongBar_train, strongBar_valid = train_test_split(strongBar_train, test_size=0.1, random_state=1926)
strongBar_train.insert(loc=7, column='func', value="train")
strongBar_valid.insert(loc=7, column='func', value="valid")
strongBar_test.insert(loc=7, column='func', value="test")

merger = pd.concat([merger_train, merger_valid, merger_test])
round = pd.concat([round_train, round_valid, round_test])
between = pd.concat([between_train, between_valid, between_test])
cigar = pd.concat([cigar_train, cigar_valid, cigar_test])
edgeOn = pd.concat([edgeOn_train, edgeOn_valid, edgeOn_test])
noBar = pd.concat([noBar_train, noBar_valid, noBar_test])
strongBar = pd.concat([strongBar_train, strongBar_valid, strongBar_test])

In [26]:
# old sum:(3134, 25481, 25045, 15620, 10381, 8993, 1375, 90029)

(3134, 25481, 25045, 15620, 10381, 8993, 1375, 90029)

In [14]:
len(merger), len(round), len(between), len(cigar), len(edgeOn), len(noBar), len(strongBar), len(merger)+len(round)+ len(between)+ len(cigar)+ len(edgeOn)+ len(noBar)+ len(strongBar)

(2174, 17480, 17024, 9937, 6902, 6087, 1291, 60895)

In [15]:
df_astre = pd.concat([merger, round, between, cigar, edgeOn, noBar, strongBar])
df_astre.to_csv("/data/renhaoye/MorCG/dataset/astre_catalog.csv")

In [16]:
df_astre

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,func,label,petro_th50,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
92711,227.337257,7.639570,15.809921,227.337304,7.639573,J150921.19+073820.9,32.051453,train,0,14.732383,...,0.323509,0.323509,0.391295,0.018277,0.013937,0.013937,1.0,dr5/png/J150/J150921.19+073820.9.png,146.227757,False
59000,247.382519,20.327953,15.649420,247.382514,20.327930,J162931.80+201940.6,14.993075,train,0,4.820569,...,0.434078,0.434078,0.431237,0.021033,0.082960,0.082960,1.0,dr5/png/J162/J162931.80+201940.6.png,140.544162,False
84505,239.859693,2.942310,15.377495,239.859706,2.942292,J155926.32+025632.3,7.544131,train,0,3.179229,...,0.244545,0.244545,0.442099,0.216582,0.070056,0.070056,1.0,dr5/png/J155/J155926.32+025632.3.png,135.608338,False
29326,153.780516,31.123194,16.464725,153.780536,31.123127,J101507.32+310723.5,16.400448,train,0,6.793319,...,0.625526,0.625526,0.260964,0.038169,0.455877,0.455877,1.0,dr5/png/J101/J101507.32+310723.5.png,156.991186,False
53468,206.828090,-0.898085,16.964060,206.828115,-0.898085,J134718.96-005354.9,5.055066,train,0,2.697565,...,0.189733,0.189733,0.651488,0.014362,0.004139,0.004139,1.0,dr5/png/J134/J134718.96-005354.9.png,115.996739,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36540,174.551235,1.040460,16.420820,174.551242,1.040463,J113812.29+010225.6,6.190662,test,6,2.659454,...,0.670500,0.670500,0.243489,0.045618,0.323277,0.323277,1.0,dr5/png/J113/J113812.29+010225.6.png,136.863078,False
118379,228.396561,27.425961,16.006632,228.396560,27.425954,J151335.17+272533.4,5.217700,test,6,2.401854,...,0.568422,0.568422,0.336576,0.044071,0.268935,0.268935,1.0,dr5/png/J151/J151335.17+272533.4.png,139.963725,False
66949,230.803584,28.309438,16.959309,230.803589,28.309446,J152312.85+281833.9,6.250730,test,6,2.777563,...,0.545725,0.545725,0.330877,0.071146,0.294163,0.294163,1.0,dr5/png/J152/J152312.85+281833.9.png,149.843297,False
93790,227.746724,6.335759,16.476753,227.746725,6.335769,J151059.21+062008.7,7.123781,test,6,2.711839,...,0.451356,0.451356,0.408462,0.077730,0.050459,0.050459,1.0,dr5/png/J151/J151059.21+062008.7.png,86.893912,False


In [17]:
overlap_dataset = match(overlap, df_astre, 2, ["del_ra", "del_dec"]).drop(columns=["del_ra", "del_dec"])

In [26]:
threshold = 0.5
threshold2 = 0.6
overlap_merger = overlap_dataset.query("label == 0").drop(columns=["func"])
overlap_round = overlap_dataset.query("label == 1").drop(columns=["func"])
overlap_between = overlap_dataset.query("label == 2").drop(columns=["func"])
overlap_cigar = overlap_dataset.query("label == 3").drop(columns=["func"])
overlap_edgeOn = overlap_dataset.query("label == 4").drop(columns=["func"])
overlap_noBar = overlap_dataset.query("label == 5").drop(columns=["func"])
overlap_strongBar = overlap_dataset.query("label == 6").drop(columns=["func"])


overlap_merger_train, overlap_merger_valid = train_test_split(overlap_merger, test_size=0.1, random_state=1926)
overlap_merger_train.insert(loc=7, column='func', value="train")
overlap_merger_valid.insert(loc=7, column='func', value="valid")

overlap_round_train, overlap_round_valid = train_test_split(overlap_round, test_size=0.1, random_state=1926)
overlap_round_train.insert(loc=7, column='func', value="train")
overlap_round_valid.insert(loc=7, column='func', value="valid")

overlap_between_train, overlap_between_valid = train_test_split(overlap_between, test_size=0.1, random_state=1926)
overlap_between_train.insert(loc=7, column='func', value="train")
overlap_between_valid.insert(loc=7, column='func', value="valid")

overlap_cigar_train, overlap_cigar_valid = train_test_split(overlap_cigar, test_size=0.1, random_state=1926)
overlap_cigar_train.insert(loc=7, column='func', value="train")
overlap_cigar_valid.insert(loc=7, column='func', value="valid")

overlap_edgeOn_train, overlap_edgeOn_valid = train_test_split(overlap_edgeOn, test_size=0.1, random_state=1926)
overlap_edgeOn_train.insert(loc=7, column='func', value="train")
overlap_edgeOn_valid.insert(loc=7, column='func', value="valid")

overlap_noBar_train, overlap_noBar_valid = train_test_split(overlap_noBar, test_size=0.1, random_state=1926)
overlap_noBar_train.insert(loc=7, column='func', value="train")
overlap_noBar_valid.insert(loc=7, column='func', value="valid")

overlap_strongBar_train, overlap_strongBar_valid = train_test_split(overlap_strongBar, test_size=0.1, random_state=1926)
overlap_strongBar_train.insert(loc=7, column='func', value="train")
overlap_strongBar_valid.insert(loc=7, column='func', value="valid")

overlap_merger = pd.concat([overlap_merger_train, overlap_merger_valid])
overlap_round = pd.concat([overlap_round_train, overlap_round_valid])
overlap_between = pd.concat([overlap_between_train, overlap_between_valid])
overlap_cigar = pd.concat([overlap_cigar_train, overlap_cigar_valid])
overlap_edgeOn = pd.concat([overlap_edgeOn_train, overlap_edgeOn_valid])
overlap_noBar = pd.concat([overlap_noBar_train, overlap_noBar_valid])
overlap_strongBar = pd.concat([overlap_strongBar_train, overlap_strongBar_valid])

In [27]:
len(overlap_merger), len(overlap_round), len(overlap_between), len(overlap_cigar), len(overlap_edgeOn), len(overlap_noBar), len(overlap_strongBar), len(overlap_merger)+len(overlap_round)+ len(overlap_between)+ len(overlap_cigar)+ len(overlap_edgeOn)+ len(overlap_noBar)+ len(overlap_strongBar)

(55, 435, 452, 263, 157, 152, 30, 1544)

In [29]:
overlap_dataset_astre = pd.concat([overlap_merger, overlap_round, overlap_between, overlap_cigar, overlap_edgeOn, overlap_noBar, overlap_strongBar])
overlap_dataset_astre.to_csv("/data/renhaoye/MorCG/dataset/overlap_dataset.csv")

In [30]:
func = ["train", "valid"]
dataset = [overlap_dataset_astre.query("func == 'train'"), overlap_dataset_astre.query("func == 'valid'")]
for i in range(len(dataset)):
    with open("/data/renhaoye/MorCG/dataset/overlap_%s.txt" % func[i], "w") as w:
        for j in range(len(dataset[i])):
            label = str(dataset[i].iloc[j].label)
            ra, dec = str(dataset[i].iloc[j, 0]), str(dataset[i].iloc[j, 1]),   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
            path = "/data/renhaoye/MorCG/dataset/out_decals/scaled/" + ra + "_" + dec + ".fits"
            if os.path.exists(path):
                w.write(path + " " + str(label)[0] +"\n")
            else:
                print(path.split(".fits")[0] + "_shifted.fits")

In [31]:
func = ["train", "valid", "test"]
dataset = [df_astre.query("func == 'train'"), df_astre.query("func == 'valid'"), df_astre.query("func == 'test'")]
for i in range(len(dataset)):
    with open("/data/renhaoye/MorCG/dataset/%s.txt" % func[i], "w") as w:
        for j in range(len(dataset[i])):
            label = str(dataset[i].iloc[j].label)
            ra, dec = str(dataset[i].iloc[j, 0]), str(dataset[i].iloc[j, 1]),   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
            path = "/data/renhaoye/MorCG/dataset/in_decals/agmtn/" + ra + "_" + dec + ".fits"
            if os.path.exists(path):
                w.write(path + " " + str(label)[0] +"\n")
            if func[i] == "train":
                if os.path.exists(path.split(".fits")[0] + "_flipped.fits"):
                    w.write(path.split(".fits")[0] + "_flipped.fits " + str(label)[0] +"\n")
                else:
                    print(path.split(".fits")[0] + "_flipped.fits")
                if os.path.exists(path.split(".fits")[0] + "_rotated.fits"):
                    w.write(path.split(".fits")[0] + "_rotated.fits " + str(label)[0] +"\n")
                else:
                    print(path.split(".fits")[0] + "_rotated.fits")
                if os.path.exists(path.split(".fits")[0] + "_shifted.fits"):
                    w.write(path.split(".fits")[0] + "_shifted.fits " + str(label)[0] +"\n")
                else:
                    print(path.split(".fits")[0] + "_shifted.fits")